In [1]:
import napari
import skimage as ski
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import glob
from _sam2_wrapper import sam2_wrapper

%load_ext autoreload
%autoreload 2

In [2]:
sam = sam2_wrapper()

In [3]:
#img = np.array([ski.io.imread(f) for f in glob.glob('U:/smc/public/SMC/sam2/datasets/cnidocyte/*.jpg')])
img = ski.io.imread('U:/smc/public/SMC/sam2/plugin/data/4x.tif')

In [4]:
ds_img = ski.transform.downscale_local_mean(img, (1, 1, 1, 1), cval=0)

In [5]:
ds_img.shape

(112, 5, 512, 512)

In [6]:
viewer = napari.Viewer()

In [7]:
viewer.add_image(img, channel_axis=1, visible=[False, True, True, True, True])

[<Image layer 'Image' at 0x22e13daeb10>,
 <Image layer 'Image [1]' at 0x22e847c59a0>,
 <Image layer 'Image [2]' at 0x22e848d4920>,
 <Image layer 'Image [3]' at 0x22e848fa990>,
 <Image layer 'Image [4]' at 0x22e13dcdb80>]

In [8]:
img = viewer.layers.selection.active.data
sam.set_image(img)

frame loading (JPEG): 100%|██████████| 112/112 [00:03<00:00, 31.81it/s]
D:\segment-anything-2\sam2\modeling\backbones\hieradet.py:68: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  x = F.scaled_dot_product_attention(


In [10]:
viewer.layers.selection.active.name

'Image [1]'

In [13]:
cur_name = viewer.layers.selection.active.name
cur_idx = -1
for id, layer in enumerate(viewer.layers):
    if layer.name == cur_name:
        cur_idx = id
        break

In [9]:
dummy_shape = np.array([[0,0,0], [0,0,1], [0, 1, 1], [0,1,0]])
viewer.add_shapes([dummy_shape], ndim=3, name='Box')

<Shapes layer 'Box' at 0x22dfea59640>

In [28]:
fnames = glob.glob('U:/smc/public/SMC/sam2/plugin/data/*.tif')
file_index = 0
file_name = fnames[file_index]
out_name = file_name.replace('.tif', '_labels.tiff')

while os.path.exists(out_name):
    file_index += 1
    if file_index >= len(fnames):
        break
    file_name = fnames[file_index]
    out_name = file_name.replace('.tif', '_labels.tiff')

In [30]:
def get_next_undone(file_index):
    file_name = fnames[file_index]
    out_name = file_name.replace('.tif', '_labels.tiff')

    while os.path.exists(out_name):
        file_index += 1
        if file_index >= len(fnames):
            break
        file_name = fnames[file_index]
        out_name = file_name.replace('.tif', '_labels.tiff')
    return file_index

In [35]:
fnames = glob.glob('S:/micro/pat/rf2602/051925_reimaged40X_YH2AX_8OHdG_Splotch_Tcof1_818240/Fused/*.tif')

In [36]:
img = ski.io.imread(fnames[0])

In [37]:
img.shape

(41, 4283, 4279, 3)

In [29]:
file_index

2

In [11]:
if viewer.layers['Box'] is not None:
    shapes = viewer.layers['Box'].data
#mask = sam.infer_from_box_single_object(shapes)
mask = sam.infer_from_box(shapes[1:], do_reset=True)
viewer.add_labels(mask.astype(int)*3)

propagate in video: 100%|██████████| 65/65 [00:04<00:00, 16.04it/s]


<Labels layer 'Labels' at 0x22dfea583e0>

In [34]:
if viewer.layers['Points'] is not None:
    good_points = viewer.layers['Points'].data
    bad_points = viewer.layers['Points [1]'].data
mask = sam.infer_from_points(good_points, bad_points, do_reset=False)
viewer.add_labels(mask.astype(int)*3)

D:\segment-anything-2\sam2\sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (D:\segment-anything-2\sam2\__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 15/15 [00:00<00:00, 17.30it/s]


<Labels layer 'Labels [3]' at 0x1d81a7e1220>

D:\segment-anything-2\sam2\sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (D:\segment-anything-2\sam2\__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 19/19 [00:00<00:00, 23.26it/s]


<Labels layer 'Labels [3]' at 0x1c1d0b1e900>